In [43]:
#intall goatools
#pip install goatools

from goatools import obo_parser
#!pip install wget
import wget
import os

#download an obo file containing all the go terms, obo is a file to handle annotation data
#use the webget library and store in a folder go_data

#go_obo_url = 'http://purl.obolibrary.org/obo/go/go-basic.obo'
#go_obo = wget.download(go_obo_url,'go_data/go-basic.obo')

#if you've already downloaded the file you can read the file directly

go_obo = 'go_data/go-basic.obo'
go = obo_parser.GODag(go_obo) #create a dictionary of GO terms

#testing the parser
#go_id = 'GO:0005858'
#go_term = go[go_id]
#print(go_term)

#testing example to make lists of go terms and descriptions containing a search query in them

vcount = []
vname = []

#replace with the search query term of your choice
vquery = 'dynein'
for go_term in go.values():
    if vquery in go_term.name:
        vcount.append(go_term.id)
        vname.append(go_term.name)
        
print('Number of GO terms with query term in their name: {}'.format(len(vcount)))

#make a dataframe of Go terms with their corresponding descriptors 

import pandas as pd
df = pd.DataFrame({'GO_TERM': vcount, 'GO_DESC': vname})

#writing df as an excel file to better analyze the output

#df.to_excel(vquery+'.xlsx', index=False)  # Specify index=False to exclude row numbers in Excel

#writing a code to loop through the GO term ids and return the gene list for each term

##getting corresponding gene names for GO term annotations
#converting the GO annotation column into a string and adding an additional column with these values 

annot = pd.read_csv('MM_ygb98ktq/out_emapper-2_annotations.csv')
dfGOs = annot['GOs']

dfGOst = []
for go in dfGOs:
    dfGOst.append(str(go))

annot['GOstring'] = dfGOst

#split the set of strings of the GO terms to analyze individual GO terms
annot['GOstring'] = annot['GOstring'].str.split(',')
#print(annot['GOstring'].head())

# Initialize a dictionary to store values of 'query' for each value in 'vcount' - the list of GO terms generated
gonames = {vc: '' for vc in vcount}

#list(gonames.items())[:4]

# Iterate over each row in the DataFrame
for index, row in annot.iterrows():
    # Iterate over each value in the 'GOstring' column of the current row
    for gostr_val in row['GOstring']:
        #If the GO value is in the stored list of GO terms, store the corresponding value of query
        if gostr_val in vcount:
            gonames[gostr_val] += row['query'] + ','

#list of GO terms for the search term =/= length of list of terms retrieved from our annotation csv file
#len(list(gonames.items()))

go_data/go-basic.obo: fmt(1.2) rel(2024-03-28) 45,727 Terms
Number of GO terms with query term in their name: 18


In [45]:
#creating a dataframe with all the requisite information
# Convert the dictionary to a DataFrame
result_df = pd.DataFrame.from_dict(gonames, orient='index', columns=['Proteins'])

# Reset the index to make 'GO Terms' a column
result_df.reset_index(inplace=True)

# Rename the columns to appropriate names
result_df.rename(columns={'index': 'GO Terms'}, inplace=True)

#add the column that gives GO term description
term_desc = []

# Iterate over each term in the 'GO Terms' column of result_df
for term in result_df['GO Terms']:
    # Iterate over each go_term in the 'GO_TERM' column of df - recall the earlier dataframe 
    for i, go_term in enumerate(df['GO_TERM']):
        # Check if the term matches the go_term
        if term == go_term:
            # Append the ith row of the other column corresponding to the matched term
            term_desc.append(df['GO_DESC'][i])
            
#there are some redundant values so converting into a set then reconverting into a list
unique_desc= list(set(term_desc))

# Insert the term description column at position 1
result_df.insert(loc=1, column='GO Description', value=unique_desc)
print(result_df.head())

# Save the modified DataFrame as a spreadsheet
#result_df.to_excel(vquery+'GO.xlsx', index=False)  # Specify index=False to exclude row numbers in Excel


     GO Terms                     GO Description  \
0  GO:0005858                     dynein complex   
1  GO:0005868           dynein axonemal particle   
2  GO:0030286   axonemal dynein complex assembly   
3  GO:0030989  dynein intermediate chain binding   
4  GO:0036156                   outer dynein arm   

                                               Genes  
0  SteCoe_266,SteCoe_12657,SteCoe_12680,SteCoe_12...  
1  SteCoe_1165,SteCoe_1842,SteCoe_2013,SteCoe_337...  
2  SteCoe_266,SteCoe_1165,SteCoe_1842,SteCoe_2013...  
3  SteCoe_10486,SteCoe_17213,SteCoe_17610,SteCoe_...  
4  SteCoe_266,SteCoe_14921,SteCoe_15463,SteCoe_16...  
